In [ ]:
geoType = {
    "esriGeometryNull" : "Unknown type of geometry",
    "esriGeometryPoint" : "Point",
    "esriGeometryMultipoint" : "Point",
    "esriGeometryLine" : "Line",
    "esriGeometryCircularArc" : "Line",
    "esriGeometryEllipticArc" : "Line",
    "esriGeometryBezier3Curve"  : "Line",
    "esriGeometryPath" : "Line",
    "esriGeometryPolyline" : "Line",
    "esriGeometryRing" : "Polygon",
    "esriGeometryPolygon" : "Polygon",
    "esriGeometryEnvelope" : "Polygon",
    "esriGeometryAny" : "Any valid geometry",
    "esriGeometryBag" : "GeometryBag",
    "esriGeometryMultiPatch" : "MultiPatch",
    "esriGeometryTriangleStrip" : "TriangleStrip",
    "esriGeometryTriangeFan" : "TriangleFan",
    "esriGeometryRay" : "Ray",
    "esriGeometrySphere" : "Polygon",
    "esriGeometryTriangles" : "Polygon"
}

In [ ]:
def cleanhtml(raw_html):
    if raw_html is not None:
        try:
            cleanr = re.compile('<.*?>')
            cleantext = re.sub(cleanr, '', raw_html)
            return cleantext
        except Exception as err:
            print(str(err))
            print(raw_html)
    return ""

In [ ]:
def download_url(in_service):
    layer_index = in_service.layers[0].url.split("/")[-1]
    url = r'https://opendata.arcgis.com/datasets/{0}_{1}.zip'.format(in_service.id, layer_index)

    try:
        r = requests.head(url)
        if r.status_code == 202:
            print(in_service.id + ": Download URL initiated: ", url)
        elif r.status_code != 200:
            print(in_service.id + "Download URL invalid: ", url + ", Status Code:", r.status_code)
            url = "error"
    except requests.ConnectionError:
        print(in_service.id + "Download URL invalid: ", url + ", Status Code:", r.status_code)
        url = "error"
    
#     print(url)
    return url

In [ ]:
def make_refs(in_service):
    ref_dict = {}
    if in_service.type == 'Feature Service':
        ref_dict[r"urn:x-esri:serviceType:ArcGIS#FeatureLayer"] = in_service.layers[0].url
        url = download_url(in_service)
        if url != "error":
            ref_dict["http://schema.org/downloadUrl"] = url
    elif in_service.type == 'Image Service':
        ref_dict[r"urn:x-esri:serviceType:ArcGIS#FeatureLayer"] = in_service.url
        if 'Albemarle Aerials' in in_service.title:            
            ref_dict["http://schema.org/downloadUrl"] = 'https://geoportal.lib.virginia.edu/UVAImageDiscovery/'
        elif 'Sanborn_' in in_service.title:  
            ref_dict["http://schema.org/downloadUrl"] = 'https://geoportal.lib.virginia.edu/UVASanbornDiscovery/'
            
    return str(ref_dict).replace("'", "\"")

In [ ]:
def bbox(extent):
    w = round(extent[0][0], 3)
    e = round(extent[1][0], 3)    
    n = round(extent[1][1], 3)    
    s = round(extent[0][1], 3)  
    
    return "ENVELOPE({0},{1},{2},{3})".format(w,e,n,s)

# bbox(service.extent)

In [ ]:
def build_json(in_services, in_places):
    solr_json = []

    for service in in_services:
        dict = {} 
        dc_format_s = "Shapefile"
        
        try:
            if not service.layers:
                print(service.id + ": Error getting service URL. Fix service")
                continue
#             if 'Table' in service.typeKeywords:
#                 print(service.id + ": Service is a table, not a Feature Class")
#                 continue
#             else:

            if service.type == 'Feature Service':
                dict["dc_type_s"] = "Dataset"
                try:
                    dict["layer_geom_type_s"] = geoType[service.layers[0].properties.geometryType]
                except:
                    print("geoType Exception")
                    if 'Table' in service.typeKeywords:
                        print(service.id + ": Service is a table, not a Feature Class")
                        
                    continue
            elif service.type == 'Image Service':
                dict["dc_type_s"] = "Image"
                dict["layer_geom_type_s"] = "Image"
                if 'Albemarle Aerials' in service.title or 'Sanborn_' in service.title:
                    dc_format_s = "Web Application"
                else:
                    dc_format_s = "Raster Dataset"
                    
            dict["dc_format_s"] = dc_format_s
            dict["geoblacklight_version"] = "1.0"
            dict["dc_description_s"] = cleanhtml(service.description)
            dict["dc_identifier_s"] = r'http://gis.lib.virginia.edu/' + service.id
            dict["layer_slug_s"] = slugify(service.title)
            dict["layer_id_s"] = "urn:urn-" + service.id    
            dict["dc_title_s"] = service.title
            dict["dc_subject_sm"] = service.tags
            dict["dc_rights_s"] = "Public"
            dict["dct_provenance_s"] = "UVa"
            dict["solr_geom"] = bbox(service.extent)
            
            if in_places:
                dict["dct_spatial_sm"] = in_places
            else:
                dict["dct_spatial_sm"] = ['Virginia']
                
            dict["dct_references_s"] = make_refs(service)
            dict["solr_year_i"] = datetime.datetime.fromtimestamp(service.created / 1e3).strftime('%Y')

            dict["layer_modified_dt"] = datetime.datetime.fromtimestamp(service.modified / 1e3).strftime('%Y-%m-%dT%H:%M:%SZ')

            solr_json.append(dict)
        except Exception as err:
            print(service.id + ": " + str(err))
            
    return(solr_json)

In [ ]:
def solr_admin(solr_str, is_json):

    if is_json:
        URL = "http://gbl-dev.scholarslab.org/solr/geoblacklight/update/json/docs?commit=true"
        headers = {'content-type': 'application/json'}
        payload = json.loads(solr_str)
#         print (payload)
        r = requests.post(URL, json=payload, headers=headers)
    else:    
        URL = "https://gbl-dev.scholarslab.org/solr/geoblacklight/update?commit=true"
#         print(solr_str)
        headers = {'content-type': 'application/xml'}    
        r = requests.post(URL, data=solr_str, headers=headers)
        
    print(r)
    print(r.text)

In [ ]:
def fetch_services(owner, in_gis, max_items):
    if not max_items:
        max_items = 99999
    
#     print(max_items)
    query = "owner:" + owner
    results = in_gis.content.search(query=query,max_items=max_items)
    
    return results

In [ ]:
## Main
from arcgis.gis import GIS
from arcgis import features
import re, json, arcpy, shutil, os, datetime, requests
from slugify import slugify


# delete_all = True
solr_json_list = []
gis = GIS('https://www.arcgis.com') #,cred[0],cred[1])

owner_dict = {'UVa_Data':[],'CharlottesvilleAdmin':['Charlottesville', 'Virginia']}

# if delete_all:
#     solr_del = '<delete><query>*:*</query></delete>'
#     solr_admin(solr_del, False)

for owner, places in owner_dict.items():
    services = fetch_services(owner, gis, None)
    solr_json = build_json(services, places)   
    
    if solr_json:
        solr_json_list = solr_json_list + solr_json
            
#         solr_json_string = json.dumps(solr_json)
#         print (solr_json_string)
        # solr_json.append(jsonstr)
        # solr_json_string
#         solr_admin(solr_json_string, True)
    else:
        print("List Empty")
  

if len(solr_json_list) > 0:
    with open(r'C:\Users\aam6g\Documents\uvadata.json','w') as outfile:
        json.dump(solr_json_list, outfile)
# print(solr_json_list)

# query = "owner:slabgis_uvalibrary AND title:VA Cities"  
# services = gis.content.search(query=query, item_type = "Feature Service", max_items=5)
# # services = gis.content.search(query=query, max_items=999)

# services

# update_solr(services, False)

In [ ]:
#*********************************************Junk**************************************************************

In [ ]:
if len(solr_json_list) > 0:
    with open(r'C:\Users\aam6g\Documents\uvadata.json','w') as outfile:
        json.dump(solr_json_list, outfile)

In [ ]:
fetch_services("UVa_Data", gis, 5)

In [ ]:
len(solr_json_list)

In [ ]:
solr_json_string = json.dumps(solr_json_list)
print (solr_json_string)

In [ ]:
update_solr(services, False)

In [ ]:
services[2].tags[-1]

In [ ]:
vars(services[2])

In [ ]:
layer = gis.content.get('710a311a241a4f67b6a36456388c2491')

In [ ]:
vars(layer)

In [ ]:
layer

In [ ]:
def update_solr(in_services, delete_all):
    if delete_all:
        solr_del = '<delete><query>*:*</query></delete>'
        solr_admin(solr_json_string, False)

#     solr_json = build_json(in_services)
    solr_json_string = json.dumps(solr_json)
    # solr_json.append(jsonstr)
    # solr_json_string
    solr_admin(solr_json_string, True)

In [ ]:
solr_del = '<delete><query>*:*</query></delete>'
solr_admin(solr_del, False)

In [ ]:
solr_json = build_json(services,[])

In [ ]:
solr_json_string = json.dumps(solr_json)
# solr_json.append(jsonstr)
# solr_json_string
solr_admin(solr_json_string, True)

In [ ]:
solr_json

In [ ]:
layer.update(categories=places)

In [ ]:
vars(layer)

In [ ]:
jsonstr= r"""
{
    "uuid": "1554988372481r8985845076931508",
    "dc_identifier_s": "1554988372481r8985845076931508",
    "dc_title_s": "U.S. National Atlas Cities 2",
    "dc_description_s": "U.S. National Atlas Cities represents cities and towns in the United States.",
    "dc_rights_s": "Public",
    "dct_provenance_s": "Esri",
    "dct_references_s": "{\"http://www.opengis.net/def/serviceType/ogc/wms\":\"http://gis.lib.virginia.edu/geoserver/wms\",\"http://www.opengis.net/def/serviceType/ogc/wfs\":\"http://gis.lib.virginia.edu/geoserver/wfs\",\"http://www.opengis.net/def/serviceType/ogc/wcs\":\"http://gis.lib.virginia.edu/geoserver/wcs\",\"http://schema.org/downloadUrl\":\"http://gis.lib.virginia.edu/geoserver/ows?service=WFS\\u0026typeName=U.S. National Atlas Cities\\u0026request=GetFeature\\u0026outputFormat=shape-zip\",\"http://www.isotc211.org/schemas/2005/gmd/\":\"https://opengeometadata.github.io/etadata.xml\"}",
    "layer_id_s": "U_S__National_Atlas_Cities",
    "layer_slug_s": "uva-U_S__National_Atlas_Cities2",
    "layer_geom_type_s": "Raster",
    "dc_format_s": "Shapefile",
    "dc_language_s": "eng",
    "georss_polygon_s": "17.712475 -176.658056 71.290556 -176.658056 71.290556 178.8775 17.712475 178.8775 17.712475 -176.658056",
    "solr_geom": "ENVELOPE(-176.658056, 178.8775, 71.290556, 17.712475)",
    "georss_box_s": "17.712475 -176.658056 71.290556 178.8775",
    "layer_modified_dt": "2016-10-22T00:00:00Z",
    "dc_type_s": "Dataset",
    "dc_subject_sm": [
        "Location",
        "Elevation",
        "Society"
    ],
    "dc_spatial_sm": [
        "<keyword>\n                        <gco:CharacterString>Downloadable Data</gco:CharacterString>\n                    </keyword>"
    ],
    "dct_issued_s": "",
    "dct_temporal_sm": "\n   ",
    "solr_year_i": 0
}
"""

solr_json_string = json.dumps(jsonstr)
json_payload(solr_json_string, True)

In [ ]:
jsonstr = r"""
{
  "dc_description_s": "Virginia Cities",
  "dc_identifier_s": "uva-drew-test-cities",
  "dc_rights_s": "Public",
  "dc_title_s": "VA Cities",
  "dc_type_s": "Dataset",
  "dct_issued_s": "2015-02-28T18:53:27Z",
  "dct_provenance_s": "UVa",
  "dct_references_s": "{\"urn:x-esri:serviceType:ArcGIS#FeatureLayer\":\"https://services2.arcgis.com/8k2PygHqghVevhzy/arcgis/rest/services/VA_Cities/FeatureServer/1\",\"http://www.isotc211.org/schemas/2005/gmd\":\"http://people.virginia.edu/~aam6g/VA_Cities_Translated.xml\"}",
  "dct_spatial_sm": [
    "Virginia"
  ],
  "layer_geom_type_s": "Point",
  "layer_id_s": "urn:urn-ad0e6ebc-824e-4450-a0d9-987f2232724f",
  "layer_modified_dt": "2016-07-18T15:17:46Z",
  "layer_slug_s": "uva-drew-test-cities",
  "solr_geom": "ENVELOPE(-83.751500, -74.973400, 39.406300, 36.434300)",
  "solr_year_i": 2015
},
{
  "dc_description_s": "Virginia Counties",
  "dc_identifier_s": "uva-drew-test-counties",
  "dc_rights_s": "Public",
  "dc_title_s": "VA Counties",
  "dc_type_s": "Dataset",
  "dct_issued_s": "2015-02-28T18:53:27Z",
  "dct_provenance_s": "UVa",
  "dct_references_s": "{\"urn:x-esri:serviceType:ArcGIS#FeatureLayer\":\"https://services2.arcgis.com/8k2PygHqghVevhzy/ArcGIS/rest/services/VA_Counties_Generalized/FeatureServer/0\",\"http://www.isotc211.org/schemas/2005/gmd\":\"http://people.virginia.edu/~aam6g/VA_Counties.xml\",\"http://schema.org/downloadUrl\":\"https://uvalibrary.maps.arcgis.com/sharing/rest/content/items/de8f397f859141c190558f33ad0643e9/data\"}",
  "dct_spatial_sm": [
    "Virginia"
  ],
  "layer_geom_type_s": "Polygon",
  "layer_id_s": "urn:urn-ad0e6ebc-824e-4450-a0d9-987f2232724f",
  "layer_modified_dt": "2016-07-18T15:17:46Z",
  "layer_slug_s": "uva-drew-test-counties",
  "solr_geom": "ENVELOPE(-83.751500, -74.973400, 39.406300, 36.434300)",
  "solr_year_i": 2015,
  "dc_format_s": "Shapefile"
}
"""
print(jsonstr)
solr_json_string = json.dumps(jsonstr)
print(solr_json_string)
# json_payload(solr_json_string, True)
